In this notebook, we provide a step-by-step walkthrough of the SPIDEC algorithm.

### 1. General Procedure

The general procedure of SPIDEC consists of four steps. Given a time series X, we:

    1. Apply a time-delay embedding to extract the subsequences (or general TD embedding vectors);
    2. Find the transitive exclusive kNNs of each subsequence;
    3. Find the UMAP embedding of the subsequences given the kNN graph;
    4. Use HDBSCAN to find the subsequence clusters.

### 2. Extracting Subsequences / TD Embeddings

We use the following function to extract TD embeddings from the time series data. When delta_t is 1, the extracted embeddings are simply sliding window subsequences. When skip_step is also 1, we extract maximally overlapping sliding windows. 

In [4]:
import numpy as np
import torch
import torch.nn.functional as F
import typing

def extract_td_embeddings(
    arr: typing.Union[np.ndarray, torch.Tensor],
    delta_t: int,
    embedding_dim: int,
    skip_step: int,
    dim_order: str = "dpt",  # [(per-time) dim, (OG seq) position, (subseq) time]
) -> torch.tensor:
    source_tensor = torch.tensor(arr) if isinstance(arr, np.ndarray) else arr
    td_embedding = F.unfold(
        source_tensor.T.view(1, source_tensor.shape[1], 1, source_tensor.shape[0]),
        (1, embedding_dim),
        dilation=delta_t,
        stride=skip_step,
    ).view(source_tensor.shape[1], embedding_dim, -1)
    if dim_order == "dpt":
        td_embedding = td_embedding.permute(0, 2, 1)
    elif dim_order == "pdt":
        td_embedding = td_embedding.permute(2, 0, 1)
    elif dim_order == "dtp":
        pass
    elif dim_order == "ptd":
        td_embedding = td_embedding.permute(2, 1, 0)
    elif dim_order == "p_td":
        td_embedding = td_embedding.permute(2, 1, 0).flatten(-2, -1)
    else:
        raise ValueError("Invalid dim_order string!")
    return td_embedding

### 3. Transitive Exclusion kNN with MPdist

In this section, we need to go through several ideas:

    1. Exclusion radius for subsequence nearest neighbours
    2. Transitive exclusion
    3. Applying transitive exclusion to MPdist

##### Exclusion Radius and Transitive Exclusion

The idea of exclusion radius first appeared in subsequence 1-nearest neighbour setting (such as in the Matrix Profile algorithm), when we want to avoid "trivial matches" - a subsequence $A$ matching with another subsequence $B$ that almost temporally overlaps with $A$. It is not helpful if an algorithm simply tells us that the subsequence at time $t$ is similar to the subsequence at $t+1$. Therefore, we often prohibit an algorithm from reporting subsequences within temporal separation $r$ as a valid nearest neighbour. Naturally, this idea extends to the k-nearest-neighbour case as well, when we want nearest neighbours of a subsequence to be outside its temporal exclusion raidus. However, in the kNN setting, there is the added problem of two nearest neighbours of $A$, let us say $B$ and $C$, being very close temporally to each other (i.e. are trivial matches of each other). This is problematic because if we want to know how many times the time series returns to a similar state as $A$, then $B$ and $C$ most likely describe the same instance of a "return", but are counted twice. Moreover, for some subsequences, all their kNNs may describe unique recurrences, but for others, it is possible most of their kNNs are duplicated counts due to such "transitive" trivial matches, leading to inconsistencies between the meaning of kNNs for different subsequences. It is therefore important to follow a procedure which produces kNNs of subsequences without any transitive trivial matches.

##### A General Function for Exclusion 1NN

Below is a general function for computing Euclidean distance nearest neighbours with exclusion radius, using PyKeOps for hardware acceleration. We use $X_i - X_j$ to symbolically encode the pairwise subsequence distances, and $I_i - I_j$ to encode pairwise time index differences, then set subsequence distance to infinity if within exclusion radius. This is not as optimised as some subsequence-specific 1NN algorithms such as Matrix Profile, but can be generalised to non-contiguous sliding windows, such as TD embedding with delays, or feature maps of TD vectors.

In [2]:
import pykeops.torch as ktorch

def exclusion_knn_search(
    X: torch.Tensor, k: int, excl: int
) -> typing.Tuple[torch.Tensor, torch.Tensor]:
    X_i = ktorch.LazyTensor(X[:, None, :])
    X_j = ktorch.LazyTensor(X[None, :, :])
    indices = torch.arange(len(X), device=X.device).float()
    I_i = ktorch.LazyTensor(indices[:, None, None])
    I_j = ktorch.LazyTensor(indices[None, :, None])
    Diag_ij = float(excl) - (I_i - I_j).abs()
    D_ij = ((X_i - X_j) ** 2).sum(-1)
    D_ij = Diag_ij.ifelse(np.inf, D_ij)
    D, I = D_ij.Kmin_argKmin(K=k, dim=1)
    D = torch.sqrt(D)
    return D, I

##### General Transitive Exclusion kNN

While it is easy to apply exclusion radius on the input subsequence, it is much harder to do so with kNN entries, as the exact temporal regions to be excluded depends on what kNN entries are returned, and the order we apply exclusion radii changes what kNN entries are considered valid. We want to achieve the following:

    * When multiple pre-exclusion kNNs are trivial matches of each other, the closest to the input subsequence should be returned as a nearest neighbour
    * We want to obtain the same number of nearest neighbours (k) for each subsequence

It is possible to first find a large number of nearest neighbours for each subsequence, then prune them to $k$ transitive-exclusive NNs in postprocessing, but a more straightforward way is to run exclusion 1NN $k$ times, expanding the exclusion regions after each iteration.

In the code section below, we use a variable I_prev to keep track of all centres of exclusion zones, which we initialise to the position of input subsequences themselves at the start, then run a transitive_exclusion_step, which will be the same as exclusion 1NN at the start. After each iteration, we append the positions of the most recently-found nearest neighbour for each subsequence to I_prev, so in the next transitive_exclusion_step, we will also exclude a radius-r zone around the nearest neigbhours from the previous step.

In [ ]:
def knn_search(feats: torch.Tensor, k: int) -> typing.Tuple[torch.tensor, torch.tensor]:
    X_i = ktorch.LazyTensor(feats[:, None, :])
    X_j = ktorch.LazyTensor(feats[None, :, :])
    D_ij = ((X_i - X_j) ** 2).sum(-1)
    D, I = D_ij.Kmin_argKmin(K=k, dim=1)
    D = torch.sqrt(D)
    return D, I


def transitive_exclusion_step(
    feats: torch.Tensor, excl: int, I_prev: torch.Tensor = None
) -> typing.Tuple[torch.Tensor, torch.Tensor]:
    X_i = ktorch.LazyTensor(feats[:, None, :])
    X_j = ktorch.LazyTensor(feats[None, :, :])
    indices = torch.arange(len(feats), device=feats.device).float()
    if I_prev is None:
        I_prev = indices[:, None]
    I_i = ktorch.LazyTensor(I_prev[:, None, :].float())
    I_j = ktorch.LazyTensor(indices[None, :, None])
    Diag_ij = (float(excl) - (I_i - I_j).abs()).ifelse(1, 0).sum(-1) - 0.5
    D_ij = ((X_i - X_j) ** 2).sum(-1)
    D_ij = Diag_ij.ifelse(np.inf, D_ij)
    D, I = D_ij.min_argmin(dim=1)
    D = torch.sqrt(D)
    return D, torch.cat((I_prev, I), dim=-1)


def transitive_exclusion_knn_search(
    feats: torch.Tensor,
    k: int,
    excl: int,
) -> typing.Tuple[torch.Tensor, torch.Tensor]:
    if excl == 0:
        return knn_search(feats, k)
    D_list = []
    for i in range(k):
        Dk, Ik = transitive_exclusion_step(
            feats, excl, I_prev=Ik if i != 0 else None
        )
        D_list += [Dk]
    return torch.cat(D_list, dim=-1), Ik[:, 1:].long()

##### Why Transitive Exclusion is Important for Subsequence Density Estimation

When we consider a "frequent pattern" or "recurrent state" for a time series, what we really want to know is how often and how precisely the time series returns to the same state. Without transitive exclusion, we cannot accurately estimate that from a kNN graph. Imagine a racecar moving through a track (or any process going through a phase space, generating a time series), and we record the position of the car at equal time intervals. The car will regularly return to each corners on the track, but it will pass through some corners faster and some slower. Effectively, we will have higher sample rate at slower corners, and the position states at slower corners will have more trivial (coming from the same lap) matches. Therefore, without transitive exclusion, it would appear that the data is denser around slower corners, because each actual revisit will results in several mutually trivial nearest neighbour matches, whereas around faster corners, it is possible that only one position state from each revisit gets matched as a nearest neighbour. However, this is not reflecting the truth, as the car revisits every corner the same amount of times, and the frequency and exactness at which the car revisits a state should only be dependent on how closely the position states match at the most matching positions from each lap. To do so, we would have to introduce transitive exclusivity in the kNN search.

##### Transitive Exclusion kNN for MPdist / Minimal Bag-of-subsequences Distance

The following code section condenses the above steps into a single function for MPdist, which is the minimal (or qth quantile) distance between all subsequences of two time series. Here, we wish to find the MPdist kNNs for all subsequences in a longer time series, therefore we will need to further break down subsequences into subsequences of subsequences, which will be used to find the MPdist values for the "meta" subsequences.

In [3]:
def mpdist_exclusion_knn_search(
    feats: torch.Tensor,
    k: int,
    bag_size: int,
    tau: int = 1,
    skip_step: int = 1,
    quantile: float = 0,
):
    pth_smallest = int(bag_size * quantile)
    # first compute subseq knn dists
    D_s, I_s = transitive_exclusion_knn_search(feats, k, bag_size)
    # break knn dist array into sliding windows
    D_s_sets, I_s_sets = extract_td_embeddings(
        D_s, tau, bag_size, skip_step, "ptd"
    ), extract_td_embeddings(I_s.float(), tau, bag_size, skip_step, "ptd")
    # now the format is (metaseq position, subseq position, knn list)
    # now pre-allocate mpdist and index arrays
    D_m, I_m = (
        torch.zeros((len(D_s_sets), k)).float().to(feats.device),
        torch.zeros((len(D_s_sets), k)).int().to(feats.device)
        - 2
        * bag_size,  # ensure no match on initial run (can be arbitrary "impossible" value)
    )
    mask = torch.zeros_like(I_s_sets).bool().to(feats.device)
    for i in range(k):
        diff = I_m[:, None, None, :] - I_s_sets[:, :, :, None]
        mask = mask | torch.any(
            torch.abs(diff) < bag_size // skip_step,
            -1,
        )

        masked_D_s_sets = torch.where(
            ~mask, D_s_sets, torch.inf
        )  # (metaseq, subseq, knn)
        match_choice_set, knn_indices = torch.min(
            masked_D_s_sets, dim=2
        )  # (metaseq, subseq)
        # knn_indices records which knn is the current unmasked smallest per metaseq per subseq

        if quantile == 0:
            D_m[:, i], min_subseq_pos = torch.min(match_choice_set, dim=1)
            # min_subseq_pos records which subseq produces the smallest dist per metaseq
        else:
            top_d, top_i = torch.topk(
                match_choice_set, pth_smallest, dim=1, largest=False
            )  # (metaseq)
            D_m[:, i], min_subseq_pos = top_d[..., -1], top_i[..., -1]

        # we still need the kth NN indices
        min_subseq_knn_indices = knn_indices.gather(
            1, min_subseq_pos[:, None]
        )  # (metaseq)
        I_m[:, i] = (
            I_s_sets.gather(1, min_subseq_pos[:, None, None].expand(-1, -1, k))
            .squeeze()
            .gather(1, min_subseq_knn_indices)
            .squeeze()
        )  # (metaseq)

    return D_m, I_m.long()

### 4. Local Density and UMAP Embedding


### 5. Clustering with HDBSCAN